In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
model = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
model.allocate_tensors()

In [3]:
model.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [4]:
model.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [5]:
def drawkeypoints(frame,keypoints,confidence_threshhold):
    y , x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for kp in shaped:
        ky,kx,kp_conf = kp
        if kp_conf > confidence_threshhold:
            cv2.circle(frame, (int(kx),int(ky)), 4, (0,255,0), -1)

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [8]:
cap = cv2.VideoCapture(1)
while True:
    ret,frame = cap.read()
    #Reshape Image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0),192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    #set inputs and outputs
    input_details = model.get_input_details()
    output_details = model.get_output_details()
    # Make Predictions
    
    model.set_tensor(input_details[0]['index'], np.array(input_image))
    model.invoke()
    keypoints_with_score = model.get_tensor(output_details[0]['index'])
    
    draw_connections(frame, keypoints_with_score, EDGES, 0.4)
    drawkeypoints(frame,keypoints_with_score,0.4)
    
    cv2.imshow('Body Gesture',frame)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()